In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Validate a Model Post Run

After executing a model, there may be an interest in checking the values of certain inputs or outputs. Although the values may be mathematically correct and converged, the inputs may have been set resulting in an output value that you do not want or that is not physical. If an input / output is found to have an undesired value, this may prompt you to change the value of an input, discretely change the structure and/or connections of the model, or raise a warning if a variable is close to an undesired bound. The `validate` method of a system in the model hierarchy can be overwritten and used to do validation of any inputs and / or outputs. The `validate` method on a system takes in the inputs / outputs in a read-only mode and can be overwritten to do whatever post-run checks are necessary in the system.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.validate
        :noindex:
```

If we, for example, take the systems from the Sellar problem, validate methods can be added to the systems to check values after the model has converged:

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src50", get_code("openmdao.test_suite.components.sellar.SellarDis1"), display=False)
glue("code_src51", get_code("openmdao.test_suite.components.sellar.SellarDis2"), display=False)

:::{Admonition} `SellarDis1` class definition 
:class: dropdown

{glue:}`code_src50`
:::

:::{Admonition} `SellarDis2` class definition 
:class: dropdown

{glue:}`code_src51`
:::

In [ ]:
import warnings
import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDis1, SellarDis2

class ValidatedSellar1(SellarDis1):
    def validate(self, inputs, outputs):
        if outputs['y1'] > 20.0:
            raise ValueError('Output "y1" is greater than 20.')


class SellarMDA(om.Group):
    def setup(self):
        cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
        cycle.add_subsystem('d1', ValidatedSellar1(), promotes_inputs=['x', 'z', 'y2'],
                            promotes_outputs=['y1'])
        cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'],
                            promotes_outputs=['y2'])

        cycle.set_input_defaults('x', 1.0)
        cycle.set_input_defaults('z', np.array([5.0, 2.0]))

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

    def validate(self, inputs, outputs):
        if outputs['y2'] < 100.0:
            warnings.warn('Output "y2" is less than 100.')

If any of the systems in a model have a validate method that you would like to run, the hierarchy of validates can be run using the `run_validation` method. This method can be run from any system and will go down the model hierarchy relative to the system that called it and run the `validate` method on each system in the hierarchy. Once `run_model()` or `run_driver()` is finished the `run_validation` method may be run.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.run_validation
        :noindex:
```

Note that when `run_validation` is run, it will collect all of the errors and warnings raised during the `validate` methods that it calls. If there are any errors among them, it will show everything collected under a ValidationError.

If the above problem is run, for example:

In [ ]:
prob = om.Problem(model=om.Group())
prob.model.add_subsystem('cycle', SellarMDA())
prob.setup()
prob.run_model()
try:
    prob.model.run_validation()
except om.ValidationError as e:
    # Avoid printing the entire error traceback for easier viewing
    print(f'Validation Error: {e}')

If there are no errors among what is collected from the `validate` methods, `run_validation` will print out everything collected without an error.

If the above problem is edited to only have warnings with no errors:

In [ ]:
class ValidatedSellar1(SellarDis1):
    def validate(self, inputs, outputs):
        if outputs['y1'] > 20.0:
            warnings.warn('Output "y1" is greater than 20.')

prob = om.Problem(model=om.Group())
prob.model.add_subsystem('cycle', SellarMDA())
prob.setup()
prob.run_model()
prob.model.run_validation()

And finally, if there are no warnings nor errors collected from the `validate` methods, the `run_validation` will print out a simple message confirming that nothing was raised.

If the Sellar problem is run with no validate errors / warnings raised:

In [ ]:
from openmdao.test_suite.components.sellar import SellarNoDerivatives

prob = om.Problem(model=om.Group())
prob.model.add_subsystem('cycle', SellarNoDerivatives())
prob.setup()
prob.run_model()
prob.model.run_validation()